<a href="https://colab.research.google.com/github/harshalDharpure/Multimodality_Hateful_Meme/blob/main/roberat_baseline_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Modified for Hindi datasets Prompthate code

In [ ]:
import torch
import torch.nn as nn
from transformers import RobertaForSequenceClassification

class RobertaBaseModel(nn.Module):
    def __init__(self, roberta, classifier, multimodal_layer=None):
        super(RobertaBaseModel, self).__init__()
        self.text_encoder = roberta
        self.classifier = classifier
        self.multimodal_layer = multimodal_layer

    def forward(self, tokens, attention_mask, feat=None):
        output = self.text_encoder(tokens, attention_mask=attention_mask).logits
        if feat is not None and self.multimodal_layer is not None:
            multimodal_output = self.multimodal_layer(output, feat)
            logits = self.classifier(multimodal_output)
        else:
            logits = self.classifier(output)
        return logits

def build_baseline(opt):
    num_labels = opt.NUM_LABELS  # The number of output classes
    text_encoder = RobertaForSequenceClassification.from_pretrained(
        opt.MODEL_NAME,
        num_labels=num_labels,
        output_attentions=False,
        output_hidden_states=False
    )

    if opt.UNIMODAL:
        # Multimodal layer should be added here, if necessary
        multimodal_layer = None
    else:
        multimodal_layer = None

    classifier = nn.Sequential(
        nn.Linear(opt.ROBERTA_DIM, opt.MID_DIM),
        nn.ReLU(),
        nn.Dropout(opt.FC_DROPOUT),
        nn.Linear(opt.MID_DIM, num_labels)
    )

    return RobertaBaseModel(text_encoder, classifier, multimodal_layer)
